In [8]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, LpBinary, LpSolutionIntegerFeasible
import pandas as pd
import numpy as np
import itertools

In [9]:
credits = {
    'Matemática Estructural': 3,
    'Calc. Dif': 3,
    'Física 1': 3,
    'IP':3,
    'Seminario de Matemáticas': 1,
    'Calc. Int': 3,
    'Algebra Lineal 1': 3,
    'EDA':3,
    'Física 2': 3,
    'Calc. Vec': 3,
    'Algebra Abstracta 1': 3,
    'Algebra Lineal 2': 3,
    'Electiva Básica 1': 3,
    'Electiva Básica 2': 3,
    'Análisis': 3,
    'Seminario de Transición': 2,
    'Var. Compleja': 3,
    'Ecuaciones Diferenciales': 3,
    'Probabilidad (Hon)': 3,
    'Algebra Abstracta 2': 3,
    'Lógica': 3,
    'Estadistica': 3,
    'Topologia': 3,
    'Geometría Diferencial': 3,
    'Medida': 3,
    'Analisis Numérico': 3,
    'Electiva Avanzada 1': 3,
    'Electiva Avanzada 2': 3,
    'Electiva Avanzada 3': 3,
    'Práctica de Enseñanza': 3,
    'Proyecto de Grado': 3,
    'Area Menor 1': 3,
    'Area Menor 2': 3,
    'CLE 1': 3,
    'CLE 2': 3,
    'CBU 1': 2,
    'CBU 2': 2,
    'CBU 3': 2,
    'CBU 4': 2,
    'CBU 5': 2,
    'CBCC': 2,
    'Escritura Universitaria 1': 2,
    'Escritura Universitaria 2': 2,
    'Constitución y Democracia': 3
}

max_credits = {1: 18, 2: 20, 3: 20, 4: 20, 5:20, 6: 20, 7: 20, 8: 20}

prereq = {
    'Calc. Int': ['Calc. Dif'],
    'EDA': ['IP'],
    'Algebra Lineal 1': ['Calc. Dif'],
    'Física 2': ['Física 1'],
    'Calc. Vec': ['Calc. Int', 'Algebra Lineal 1'],
    'Algebra Abstracta 1': ['Algebra Lineal 1', 'Matemática Estructural'],
    'Algebra Lineal 2': ['Algebra Lineal 1'],
    'Análisis': ['Calc. Int', 'Matemática Estructural'],
    'Seminario de Transición': ['Seminario de Matemáticas'],
    'Var. Compleja': ['Calc. Vec'],
    'Ecuaciones Diferenciales': ['Calc. Vec'],
    'Probabilidad (Hon)': ['Calc. Vec'],
    'Algebra Abstracta 2': ['Algebra Abstracta 1'],
    'Lógica': ['Algebra Abstracta 1'],
    'Estadistica': ['Probabilidad (Hon)'],
    'Topologia': ['Análisis'],
    'Geometría Diferencial': ['Análisis'],
    'Medida': ['Análisis'],
    'Analisis Numérico': ['Ecuaciones Diferenciales','IP'],
    'Práctica de Enseñanza': ['Algebra Abstracta 2'],
    'Proyecto de Grado': ['Seminario de Transición']
}

coreq = {
    'Física 1': ['Calc. Dif'],
    'Física 2': ['Calc. Int'],
    'Electiva Básica 1': ['Calc. Int'],
    'Electiva Básica 2': ['Calc. Int'],
    'Electiva Avanzada 1': ['Análisis'],
    'Electiva Avanzada 2': ['Análisis'],
    'Electiva Avanzada 3': ['Análisis'],
    'Escritura Universitaria 2': ['Escritura Universitaria 1'],
}

S = 8

In [10]:
variables = {}
i = 1
for m in credits:
    for j in range(0,S+1):
        variables[(m,j)] = LpVariable(name='x_'+str(i)+','+str(j), lowBound = 0, upBound= 1, cat=LpBinary)
    i += 1
variables

{('Matemática Estructural', 0): x_1,0,
 ('Matemática Estructural', 1): x_1,1,
 ('Matemática Estructural', 2): x_1,2,
 ('Matemática Estructural', 3): x_1,3,
 ('Matemática Estructural', 4): x_1,4,
 ('Matemática Estructural', 5): x_1,5,
 ('Matemática Estructural', 6): x_1,6,
 ('Matemática Estructural', 7): x_1,7,
 ('Matemática Estructural', 8): x_1,8,
 ('Calc. Dif', 0): x_2,0,
 ('Calc. Dif', 1): x_2,1,
 ('Calc. Dif', 2): x_2,2,
 ('Calc. Dif', 3): x_2,3,
 ('Calc. Dif', 4): x_2,4,
 ('Calc. Dif', 5): x_2,5,
 ('Calc. Dif', 6): x_2,6,
 ('Calc. Dif', 7): x_2,7,
 ('Calc. Dif', 8): x_2,8,
 ('Física 1', 0): x_3,0,
 ('Física 1', 1): x_3,1,
 ('Física 1', 2): x_3,2,
 ('Física 1', 3): x_3,3,
 ('Física 1', 4): x_3,4,
 ('Física 1', 5): x_3,5,
 ('Física 1', 6): x_3,6,
 ('Física 1', 7): x_3,7,
 ('Física 1', 8): x_3,8,
 ('IP', 0): x_4,0,
 ('IP', 1): x_4,1,
 ('IP', 2): x_4,2,
 ('IP', 3): x_4,3,
 ('IP', 4): x_4,4,
 ('IP', 5): x_4,5,
 ('IP', 6): x_4,6,
 ('IP', 7): x_4,7,
 ('IP', 8): x_4,8,
 ('Seminario de Mat

In [11]:
def create_model(credits,variables, max_credits,prereq,coreq,S):
    model = LpProblem('Horario')

    # Consistency of x-es along periods
    for m in credits:
        for j in range(0,S):
            model += ( variables[(m,j)] <= variables[(m,j+1)] )

    # Zero Semester 

    for m in credits:
        model += (variables[(m,0)] == 0)

    # First Semester

    sum = lpSum([variables[(m,1)] * credits[m] for m in credits])
    model += (sum <= max_credits[1])

    # Other Semesters

    for k in range(1,S):
        sum = lpSum([(variables[(m,k+1)] - variables[(m,k)]) * credits[m] for m in credits])
        model += (sum <= max_credits[k+1])

    # Pre-requisites

    for b in prereq:
        for a in prereq[b]:
            for j in range(0,S):
                model += (variables[(a,j)] >= variables[(b,j+1)])

    # Co-requisites

    for b in coreq:
        for a in coreq[b]:
            for j in range(1,S+1):
                model += (variables[(a,j)] >= variables[(b,j)])

    # Final condition

    for m in credits:
        model += (variables[(m,S)] == 1)
    return model

In [12]:
def reduce_max_credits(max_credits, n):
    upper_bound = max(list(max_credits.values())) - n
    return {j: min(upper_bound,k) for j,k in max_credits.items()}

In [13]:
bandera = True
n = 0
while bandera:
    model = create_model(credits, variables, reduce_max_credits(max_credits, n), prereq, coreq, S)
    status = model.solve()
    if status == 1:
        best_model = model
    else:
        bandera = False
    n += 1

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8_/_cvslyx53dx_h_h6mjxq933m0000gn/T/9d2835a937c540599d9518952724d402-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/8_/_cvslyx53dx_h_h6mjxq933m0000gn/T/9d2835a937c540599d9518952724d402-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 717 COLUMNS
At line 3491 RHS
At line 4204 BOUNDS
At line 4602 ENDATA
Problem MODEL has 712 rows, 397 columns and 1980 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0002I 44 variables fixed
Cgl0004I processed model has 306 rows, 223 columns (223 integer (223 of which binary)) and 1042 elements
Cbc0045I No integer variables out of 223 objects (223 integer) have costs
Cbc0045I branch on satisfied N create fake objective Y random co

In [14]:
inv_var = {v.name: k for k, v in variables.items()}

periods = {}
for var in best_model.variables():
        if var.name in inv_var:
                m, j = inv_var[var.name]
                if var.value() == 1:
                        periods[m] = min(periods.get(m,S), j)

for j in range(1,S+1):
    print(f"{j} semestre:")
    for m in periods:
        if periods[m] == j:
            print('    ',m)

1 semestre:
     Matemática Estructural
     Calc. Dif
     Física 1
     Area Menor 1
     IP
2 semestre:
     Electiva Básica 1
     Electiva Básica 2
     Area Menor 2
     Calc. Int
     Algebra Lineal 1
3 semestre:
     Calc. Vec
     Algebra Abstracta 1
     Algebra Lineal 2
     Análisis
     Electiva Avanzada 1
4 semestre:
     Var. Compleja
     Ecuaciones Diferenciales
     Probabilidad (Hon)
     Algebra Abstracta 2
     Lógica
5 semestre:
     Topologia
     CLE 1
     CLE 2
     Constitución y Democracia
     EDA
6 semestre:
     Estadistica
     Geometría Diferencial
     Medida
     Analisis Numérico
     Seminario de Matemáticas
7 semestre:
     Seminario de Transición
     Electiva Avanzada 2
     Electiva Avanzada 3
     Práctica de Enseñanza
     CBU 1
     CBU 2
     Física 2
8 semestre:
     Proyecto de Grado
     CBU 3
     CBU 4
     CBU 5
     CBCC
     Escritura Universitaria 1
     Escritura Universitaria 2
